# Использование упреждающих сетей при NLP

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Определим глобальные переменные

In [3]:
LABELS = [0, 0, 1, 1] # обозначение множеств
CENTERS = [(-3, -3), (3, 3), (3, -3), (-3, 3)] # центры множеств

### Определяем модель

In [5]:
class MultilayerPerceptron(nn.Module):
    
    def __init__(self, input_size, hidden_size=2, output_size=3,
                 num_hidden_layers=1, hidden_activation=nn.Sigmoid):
        """
        Инициализируем веса.
        Аргументы:
            input_dim (int): размер входных векторов
            hidden_dim (int): размер выходных результатов первого линейного слоя
            output_dim (int): размер выходных результатов второго линейного слоя 
            num_hidden_layers(int): количество скрытых слоев
            hidden_activation(torch.nn.*): функция активации
        """
        
        super(MultilayerPerceptron, self).__init__()
        # модуль, содержащий список слоев
        self.module_list = nn.ModuleList()
        
        interim_input_size = input_size
        interim_output_size = hidden_size
        
        # создаем скрытые слои
        for _ in range(num_hidden_layers):
            # добавляем слой, учитывая его размер
            self.module_list.append(nn.Linear(interim_input_size, interim_output_size))
            # добавляем функцию активации
            self.module_list.append(hidden_activation())
            interim_input_size = interim_output_size
            
        self.fc_final = nn.Linear(interim_input_size, output_size)
        self.last_forward_cache = []
        
    def forward(self, x, apply_softmax=False):
        """
        Прямой проход MLP
        Аргументы:
            x_in (torch.Tensor): тензор входных данных
            Значение x_in.shape должно быть (batch, input_dim)
            apply_softmax (bool): флаг для многомерной логистической функции
            активации. При использовании функции потерь на основе
            перекрестной энтропии должен равняться false
        Возвращает:
            итоговый тензор. Значение tensor.shape должно
            быть (batch, output_dim)
        """
        self.last_forward_cache = []
        self.last_forward_cache.append(x.to('cpu').numpy())
        
        for module in self.module_list:
            x = module(x)
            self.last_forward_cache.append(x.to('cpu').data.numpy)
            
        output = self.fc_final(x)
        self.last_forward_cache.append(output.to('cpu').data.numpy())
        
        if apply_softmax:
            output = F.softmax(output, dim=1)
            
        return output

### Сгенерируем данные для обучения

In [6]:
def get_toy_data(batch_size):
    assert len(CENTERS) == len(LABELS), 'количество центров должно быть равно количеству меток'
    
    x_data = []
    y_targets = np.zeros(batch_size)
    n_centers = len(CENTERS)
    
    for batch_i in range(batch_size):
        center_idx = np.random.randint(0, n_centers)
        x_data.append(np.random.normal(loc=CENTERS[center_idx]))
        y_targets[batch_i] = LABELS[centr_idx]
        
    return torch.tensor(x_data, dtype=torch.float32), torch.tensor(y_targets, dtype=torch.int64)

### Визуализация результатов

In [8]:
def visualize_results(perceptron, x_data, y_truth, n_samples=1000, ax=None, epoch=None,
                      title='', levels=[0.3, 0.4, 0.5], linestyles=['--', '-', '--']):
    _, y_pred = perceptron(x_data, apply_softmax=True).max(dim=1)
    y_pred = y_pred.data.numpy()
    
    x_data = x_data.data.numpy()
    y_truth = y_truth.data.numpy()
    
    n_classes = len(set(LABELS))
    
    all_x = [[] for _ in range(n_classes)]
    all_colors = [[] for _ in range(n_classes)]
    
    colors = ['black', 'white']
    markers = ['o', '*']
    
    for x_i, y_pred_i, y_true_i in zip(x_data, y_pred, y_truth):
        all_x[y_true_i].append(x_i)
        if y_pred_i == y_true_i:
            all_colors[y_true_i].append('white')
        else:
            all_colors[y_true_i].append('black')
        # all_colors[y_true_i].append(colors[y_pred_i])
        
    all_x = [np.stack(x_list) for x_list in all_x]
    
    if ax is None:
        _, ax = plt.subplots(1, 1, figsize=(10, 10))
        
    for x_list, color_list, marker in zip(all_x, all_colors, markers):
        ax.scatter(x_list[:, 0], x_list[:, 1], edgecolor='black', marker=marker, facecolor=color_list, s=100)
        
    xlim = (min([x_list[:, 0].min() for x_list in all_x])), (max([x_list[:, 0].min() for x_list in all_x]))
    
    ylim = (min([x_list[:, 1].min() for x_list in all_x])), (max([x_list[:, 1].min() for x_list in all_x]))
    
    # строим гиперплоскость
    xx = np.linspace(xlim[0], xlim[1], 30)
    yy = np.linstace(ylim[0], ylim[1], 30)    
    YY, XX = np.meshgrid(yy, xx)    
    xy = np.vstack([XX.ravel(), YY.ravel()]).T
    
    for i in range(n_classes):
        Z = perceptron(torch.tensor(xy, dtype=torch.float32), apply_softmax=True)
        Z = Z[:, i].data.numpy().reshape(XX.shape)
        ax.contour(XX, YY, Z, colors=colors[i], levels=levels, linestyles=linestyles)
        
    plt.suptitle(title)
    
    if epoch is not None:
        plt.text(xlim[0], ylim[0], 'Epoch = {}'.format(str(epoch)))